In [1]:
import queue
import time
import os
import sys

from DataHandler.TradeLOBHourlyDataHandler import HistoricTradeLOBHourlyDataHandler
from Strategy.BuyAndHoldStrategy import BuyAndHoldStrategy
from Portfolio.LogPlotPortfolio import LogPlotPortfolio
from Execution.execution import SimulatedExecutionHandler

event_queue = queue.Queue()
data_handler = HistoricTradeLOBHourlyDataHandler(event_queue, 
                                           symbol_list=['btc_usdt','btc_usdt'],
                                           exchange_list=['binance','okex'], 
                                           file_dir = 'data_sample/20240101/', 
                                           # file_dir = 'data_sample/',
                                           is_csv=False)
#sys.exit()

portfolio = LogPlotPortfolio(event_queue, data_handler)      # 组合
executor = SimulatedExecutionHandler(event_queue, data_handler)   # 回测模拟成交器；如果是实盘这里就是算法交易模块
strategy = BuyAndHoldStrategy(event_queue, data_handler, portfolio, executor)       # 策略实例。实际应用中应该有多个策略实例

cnt = 0
while True:
    # Update the trade/LOB (specific backtest code, as opposed to live trading)
    if data_handler.continue_backtest == True:
        data_handler.update_TradeLOB()
    else:
        break
    
    cnt+=1
    if cnt>2000: break

    # Handle the events
    while True:
        try:
            event = event_queue.get(False)
        except queue.Empty:
            break
        else:
            if event is not None:

                if event.type == 'MARKET':
                    print('get market event', event)
                    strategy.calculate_signals(event)
                    portfolio.update_holdings_from_market()
                    executor.on_market_event(event)

                elif event.type == 'ORDER':
                    executor.on_order_event(event)

                elif event.type == 'FILL':
                    print('get fill event', event)
                    portfolio.update_positions_from_fill(event)
                    strategy.on_order_fill(event)
                    #sys.exit()

backtest on:  ['btc_usdt_binance', 'btc_usdt_okex']
/*----- start initialize the DataHandler -----*/
/*----- DataHandler initialization ends -----*/

===== reload data from new hour =====

===== processing market event in  1704042000000  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1704042000004  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1704042000006  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1704042000007  =====
get new market events and push to queue
get market event {'type': 'MARKET'}
put orders to the exexution {'type': 'ORDER', 'timestamp': 1704042000107, 'symbol': 'btc_usdt_binance', 'order_id': 0, 'order_type': 'MARKET', 'direction': 'BUY', 'quantity': 0.023478328446319266, 'price': None}

===== processing market event in  1704042000008  =====
get new market e

In [2]:
portfolio.all_holdings

{'btc_usdt_binance': {1704042000117: 1000.0,
  1704042000118: 999.9998826083577,
  1704042000126: 1000.0022304412024,
  1704042000128: 1000.0023478328445,
  1704042000130: 1000.004578274047,
  1704042000137: 1000.0054000155425,
  1704042000144: 1000.0062217570382,
  1704042000146: 1000.0096261146631,
  1704042000147: 1000.0126782973608,
  1704042000150: 1000.0130304722875,
  1704042000195: 1000.015730480059,
  1704042000202: 1000.0184304878304,
  1704042000338: 1000.0781828337263,
  1704042000339: 1000.0859306821134,
  1704042000349: 1000.0891002564537,
  1704042000421: 1000.0903915645183,
  1704042000427: 1000.0916828725829,
  1704042001027: 1000.0923872224362,
  1704042001029: 1000.0925046140785,
  1704042001030: 1000.0930915722895,
  1704042001927: 1000.1116394517621,
  1704042001928: 1000.1297177646658,
  1704042001935: 1000.1320655975105,
  1704042001936: 1000.1344134303552,
  1704042002254: 1000.1423960620269,
  1704042003193: 1000.1466221611471,
  1704042003241: 1000.15272652654

In [3]:
import pandas as pd

pd.read_parquet('data_sample/20240101/btc_usdt_binance_trade.parquet').head()

,price,qty,time,maker
0,42592.47,0.00033,1704042000000,BUY
1,42592.47,0.00187,1704042000000,BUY
2,42592.47,0.00152,1704042000000,BUY
3,42592.47,0.00175,1704042000004,BUY
4,42592.46,0.00262,1704042000004,SELL


In [4]:
import pandas as pd

pd.read_parquet('data_sample/20240101/btc_usdt_binance_LOB.parquet').head()

,time,ask1,ask1_qty,bid1,bid1_qty
0,1704042000007,42592.47,0.62603,42592.46,3.16954
1,1704042000007,42592.47,0.62603,42592.46,3.16720
2,1704042000007,42592.47,0.62560,42592.46,3.16720
3,1704042000007,42592.47,0.62560,42592.46,3.16598
4,1704042000007,42592.47,0.62455,42592.46,3.16598
